## Test Preprocessor

In [1]:
from src.text_processing import *

en_core_web_sm is already installed.


In [2]:
from src.general import *

In [3]:
# get all the files in data folder
files_to_process =  get_files_in_directory("../data/")

In [4]:
files_to_process

['../data/all_mcqs.csv',
 '../data/all_texts.csv',
 '../data/text1.txt',
 '../data/text2.txt',
 '../data/text3.txt',
 '../data/anthropology\\1.4.txt',
 '../data/anthropology\\1.5.txt',
 '../data/anthropology\\10.1.txt',
 '../data/anthropology\\11.1.txt',
 '../data/anthropology\\11.4.txt',
 '../data/anthropology\\18.2.txt',
 '../data/anthropology\\18.3.txt',
 '../data/anthropology\\18.4.txt',
 '../data/anthropology\\2.3.txt',
 '../data/anthropology\\20.1.txt',
 '../data/anthropology\\3.2.txt',
 '../data/anthropology\\3.4.txt',
 '../data/anthropology\\4.5.txt',
 '../data/anthropology\\5.1.txt',
 '../data/anthropology\\5.2.txt',
 '../data/anthropology\\6.3.txt',
 '../data/anthropology\\6.4.txt',
 '../data/anthropology\\7.3.txt',
 '../data/anthropology\\8.2.txt',
 '../data/anthropology\\8.5.txt',
 '../data/history\\1.3.txt',
 '../data/history\\10.3.txt',
 '../data/history\\10.4.txt',
 '../data/history\\11.3.txt',
 '../data/history\\11.5.txt',
 '../data/history\\13.2.txt',
 '../data/history

In [5]:
text = read_text_file('../data/text3.txt')

In [6]:
chunked_text = add_chunk_markers(text)

In [7]:
chunked_text

'<chunk1>Research magazine from Stanford University Journalist Anne Simonson â€“ January 13, 2008 The sun counteracts cancer\nSunrays that hit our skin reinforce the bodyâ€™s own defense against cancer. The campaigns to get people out of the sun have gone too far, according to the researcher Johan Moan. â€œThe rays of the sun speed up the bodyâ€™s production of vitamin D. This can prevent cancer as well as many other diseases.â€\x9d So says Johan Moan, Professor at the Cancer Center at Boston Medical Center. Along with colleagues from Norway, he is presenting results that support the theory that sun exposure provides the body with a more effective defense against cancer. The findings show, for example, that people who live closer to the equator, and whose bodies hence are more exposed to the sun, have a better chance of survival if they develop cancer in their internal organs. Moreover, they show that cancer patients have a better prognosis if they are diagnosed in summer or autumn, wh

## Test Planner

In [8]:
from src.planner import *

In [9]:
import uuid

# Generate a unique invocation ID for this workflow run
invocation_id = str(uuid.uuid4())



In [10]:
plan = await generate_plan(
    invocation_id=invocation_id,
    model="gpt-4o",
    text=chunked_text, 
    fact = 1, 
    inference = 1, 
    table_name = "plan_metadata", 
    database_file = "../database/mcq_metadata.db")

INFO:root:Table 'plan_metadata' already exists.
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': '[ROLE]:\n  You are an experienced college instructor specializing in developing reading comprehension questions to evaluate students\' understanding of academic texts, such as textbook chapters and academic articles. You excel at identifying and selecting key facts and inferences from these texts for question development.\n\n[TASK]:\n  Your task is to summarize the academic text provided by the user and select the specified number of facts and inferences as requested by the user following the guidelines below. \n\n[Guidelines]\n  Step1: Summarize the text \n  The text is divided into chunks (as indicated by the html tags such as <chunk1></chunk1>). Summarize the content within each chunk, maintaining the chunk tags in your summary.\n\n  Step2: Identify key facts and/or inference

In [11]:
plan

{'system_prompt': '[ROLE]:\n  You are an experienced college instructor specializing in developing reading comprehension questions to evaluate students\' understanding of academic texts, such as textbook chapters and academic articles. You excel at identifying and selecting key facts and inferences from these texts for question development.\n\n[TASK]:\n  Your task is to summarize the academic text provided by the user and select the specified number of facts and inferences as requested by the user following the guidelines below. \n\n[Guidelines]\n  Step1: Summarize the text \n  The text is divided into chunks (as indicated by the html tags such as <chunk1></chunk1>). Summarize the content within each chunk, maintaining the chunk tags in your summary.\n\n  Step2: Identify key facts and/or inferences\n  Depending on the user’s requests, create a list of key facts and/or inferences from the academic text. The number of facts or inferences generated in this step can be more than what the u

## Test Controller

In [12]:
from src.controller_helper import *

In [13]:
fact =1
inference = 1
main_idea = 1

In [14]:
task_list = create_task_list(chunked_text, plan, fact, inference, main_idea)

In [15]:
task_list

[{'question_type': 'fact',
  'content': "Sun exposure enhances the body's vitamin D production, which can prevent cancer and other diseases.",
  'text': 'Research magazine from Stanford University Journalist Anne Simonson â€“ January 13, 2008 The sun counteracts cancer\nSunrays that hit our skin reinforce the bodyâ€™s own defense against cancer. The campaigns to get people out of the sun have gone too far, according to the researcher Johan Moan. â€œThe rays of the sun speed up the bodyâ€™s production of vitamin D. This can prevent cancer as well as many other diseases.â€\x9d So says Johan Moan, Professor at the Cancer Center at Boston Medical Center. Along with colleagues from Norway, he is presenting results that support the theory that sun exposure provides the body with a more effective defense against cancer. The findings show, for example, that people who live closer to the equator, and whose bodies hence are more exposed to the sun, have a better chance of survival if they develo

In [16]:
task = task_list[0]
task.get("question_type", "").lower()

'fact'

### Test MCQ Generators

In [17]:
from src.mcq_generation import *

In [18]:
questions = await generate_all_mcqs(task_list, invocation_id, 
                                    model="gpt-4o", 
                                    mcq_metadata_table_name="mcq_metadata", 
                                    evaluation_metadata_table_name="evaluation_metadata",
                                    database_file="../database/mcq_metadata.db",
                                    max_attempt=3,
                                    concurrency=30,
                                    )


INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': '[ROLE]:\nYou are an experienced college instructor. You are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\n\n[TASK]:\nYour task is to write a factual multiple-choice question (see DEFINITION) based on a fact provided by the user. \nThis fact will be directly supported by a "source text", which is an excerpt from the academic material. \nA "contextual text" will also be provided to help you understand the broader meaning and useage of the source text. \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do. \nNOTE: The user may provide an already generated que

In [19]:
questions

[{'system_prompt': '[ROLE]:\nYou are an experienced college instructor. You are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\n\n[TASK]:\nYour task is to write a factual multiple-choice question (see DEFINITION) based on a fact provided by the user. \nThis fact will be directly supported by a "source text", which is an excerpt from the academic material. \nA "contextual text" will also be provided to help you understand the broader meaning and useage of the source text. \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do. \nNOTE: The user may provide an already generated question with evaluations, in which case you should revise the provided question based on the guidelines in [GUIDELINES] and the evaluations.\n\n[DEFINITION]\n**What are factual questions?**\nFactual questions are those that have a single, verifiable answer based on objective informatio

In [20]:
questions = await generate_all_mcqs_quality_first(task_list, 
                                                  invocation_id, 
                                    model="gpt-4o", 
                                    mcq_metadata_table_name="mcq_metadata", 
                                    evaluation_metadata_table_name="evaluation_metadata",
                                    ranking_metadata_table_name="ranking_metadata",
                                    database_file="../database/mcq_metadata.db",
                                    max_attempt=3,
                                    candidate_num=5,
                                    concurrency=30,
                                    )

INFO:root:Table 'ranking_metadata' already exists.
INFO:root:Table 'ranking_metadata' already exists.
INFO:root:Table 'ranking_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connect

In [21]:
questions

[{'system_prompt': '[ROLE]:\nYou are an experienced college instructor. \nYou are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\nYou have rich experience in evaluating the quality of multiple-choice questions using a rubric.\n\n[TASK]:\nThe user provides a set of multiple-choice questions addressing the same fact, inference, or main idea.\nYour task is to evaluate the quality of these multiple-choice questions and select the best question.  \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do.\n\n[GUIDELINES]\nStep1: Understand the definition of the type of question you are evaluating (factual, inferential, or main idea).\n **Fact Question**: Requires recall of specific facts or details that is explicitly stated in the source text. \n  Example: \n  What methods does ethnoarchaeology utilize to study past human cultures?\n  A) Linguistic and biological me

### Test the Quality-first MCQ generation

In [18]:
ranking_metadata = await generate_mcq_quality_first(
    invocation_id=invocation_id,
    model="gpt-4o",
    task=task_list[0],
    mcq_table_name="mcq_metadata",
    ranking_table_name="ranking_metadata",
    database_file='../database/mcq_metadata.db',
    max_attempt = 3,
    attempt= 1,
    candidate_num = 5
)


INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'ranking_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
INFO:root:Table 'mcq_metadata' already exists.
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': '[ROLE]:\nYou are an experienced college instructor. You are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\n\n[TASK]:\nYour task is to write a factual multiple-choice question (see DEFINITION) based on a fact provided by the user. \nThis fact will be directly supported by a "source text", which is an excerpt from the academic material. \nA "contextual text" will also be provided to help you understa

## Test the Formatter

In [18]:
from src.formatter import *

In [19]:
reformatted_questions = reformat_mcq_metadata(questions)

In [20]:
reformatted_questions

[{'system_prompt': '[ROLE]:\nYou are an experienced college instructor. You are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\n\n[TASK]:\nYour task is to write a factual multiple-choice question (see DEFINITION) based on a fact provided by the user. \nThis fact will be directly supported by a "source text", which is an excerpt from the academic material. \nA "contextual text" will also be provided to help you understand the broader meaning and useage of the source text. \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do. \n\n[DEFINITION]\n**What are factual questions?**\nFactual questions are those that have a single, verifiable answer based on objective information, often requiring recall of specific facts or details. \n\n[GUIDELINES]\nGenerate a multiple-choice factual question by completing the following steps in order:\n\nStep1: Based on the fact, 

## Test the Full Workflow

In [1]:
from src.general import *

# get all the files in data folder
files_to_process =  get_files_in_directory("../data/")

In [2]:
text = read_text_file('../data/text3.txt')

In [3]:
from src.workflow import *

en_core_web_sm is already installed.


c:\Users\ytian126\Documents\repos\MCQ-generation\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
output = await question_generation_workflow(text, 
                                            fact = 1, 
                                            inference = 1, 
                                            main_idea = 1, 
                                            model = "gpt-4o", 
                                            quality_first = True, 
                                            candidate_num =5,
                                            max_attempt_for_single_mcq = 3,
                                            plan_metadata_table_name = "plan_metadata",
                                            mcq_metadata_table_name = "mcq_metadata",
                                            evaluation_metadata_table_name = "evaluation_metadata",
                                            ranking_metadata_table_name = "ranking_metadata",
                                            workflow_metadata_table_name = "workflow_metadata", 
                                            database_file = "../database/mcq_metadata.db",
                                            concurrency = 10)


2025-09-13 17:14:46,166 - INFO - Workflow started
2025-09-13 17:14:46,240 - INFO - Text successfully chunked
2025-09-13 17:14:46,242 - INFO - Table 'plan_metadata' already exists.
2025-09-13 17:14:53,696 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-13 17:14:53,713 - INFO - Metadata inserted into 'plan_metadata'.
2025-09-13 17:14:53,715 - INFO - Plan generated
2025-09-13 17:14:53,716 - INFO - create_task_list called (requested: facts=1, inferences=1, main_idea=1)
2025-09-13 17:14:53,716 - INFO - validate_and_parse_plan called (expected facts=1, inferences=1)
2025-09-13 17:14:53,717 - INFO - Counts — facts: expected=1 actual=1; inferences: expected=1 actual=1
2025-09-13 17:14:53,717 - INFO - Validation passed. Parsed summary_len=2762, facts=1, inferences=1
2025-09-13 17:14:53,717 - INFO - Validated plan: facts=1, inferences=1
2025-09-13 17:14:53,718 - INFO - extract_chunks called with 1 chunk label(s)
2025-09-13 17:14:53,719 - INFO - ex

In [5]:
print(output)

[{'system_prompt': '[ROLE]:\nYou are an experienced college instructor. \nYou are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\nYou have rich experience in evaluating the quality of multiple-choice questions using a rubric.\n\n[TASK]:\nThe user provides a set of multiple-choice questions addressing the same fact, inference, or main idea.\nYour task is to evaluate the quality of these multiple-choice questions and select the best question.  \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do.\n\n[GUIDELINES]\nStep1: Understand the definition of the type of question you are evaluating (factual, inferential, or main idea).\n **Fact Question**: Requires recall of specific facts or details that is explicitly stated in the source text. \n  Example: \n  What methods does ethnoarchaeology utilize to study past human cultures?\n  A) Linguistic and biological me

In [2]:
# Export the output from database to a CSV file 
from src.database_handler import export_table_to_csv

In [6]:
export_table_to_csv(
    table_name="mcq_metadata",
    file_name="20250530_mcq_metadata.csv",
    database_file="../database/mcq_metadata.db",

)


Exported 25 rows to ../output\20250530_mcq_metadata.csv


In [ ]:
{'system_prompt': '...',
  'user_prompt': '...',
  'model': 'gpt-4o',
  'completion': "Step 1: **Understanding of Inferential Questions**\nInferential questions require the reader to read between the lines and make logical connections that aren't directly stated but are supported by the text. They ask students to infer or deduce information by applying their reasoning skills to the given evidence in the text.\n\nStep 2: **Correct Answer**\nBased on the inference and the source text, the correct answer involves understanding how avoiding sun exposure entirely, especially in regions with limited sunlight, can lower vitamin D levels, which in turn leads to health issues. Vitamin D is synthesized in the skin upon exposure to sunlight and is important for various bodily functions.\n\n<ANSWER>B) Insufficient production of vitamin D can lead to health issues</ANSWER>\n\nStep 3: **Plausible Distractors**\n1. Avoiding sun exposure increases the risk of skin cancer.\n2. Using sunscreen always results in low vitamin D synthesis.\n3. Sun exposure guarantees protection against all internal cancers.\n\nStep 4: **Shuffle Options**\nNow, I'll shuffle the correct answer with the distractors to create a set of answer choices.\n\nStep 5: **Self-Critique and Ensure Quality**\n- The question is clear and asks the reader to infer the effects of entirely avoiding sun exposure.\n- The correct answer relates directly to vitamin D's role as implied in the source text.\n- The distractors are plausible but incorrect interpretations based on common misconceptions and not directly supported by the text.\n- The answer choices are concise and similar in length to maintain consistency.\n\n<QUESTION>What can be inferred as a potential health risk of completely avoiding sun exposure, especially in regions with limited sunlight?  \n  \\nA) Avoiding sun exposure increases the risk of skin cancer.\n  \\nB) Insufficient production of vitamin D can lead to health issues.\n  \\nC) Using sunscreen always results in low vitamin D synthesis.\n  \\nD) Sun exposure guarantees protection against all internal cancers.</QUESTION>",
  'execution_time': '0:00:10.742509',
  'input_tokens': 1093,
  'output_tokens': 394,
  'question_type': 'inference',
  'invocation_id': 'e59db387-b17c-46b7-9f06-97b7f24edfbe',
  'chunk': '["chunk1", "chunk2"]',
  'mcq': 'Q2: What can be inferred as a potential health risk of completely avoiding sun exposure, especially in regions with limited sunlight?  \n  \nA) Avoiding sun exposure increases the risk of skin cancer.\n  \nB) Insufficient production of vitamin D can lead to health issues.\n  \nC) Using sunscreen always results in low vitamin D synthesis.\n  \nD) Sun exposure guarantees protection against all internal cancers.',
  'mcq_answer': 'Q2: B) Insufficient production of vitamin D can lead to health issues',
}

In [ ]:
x = {
  "reasoning": "Your reasoning about the shortening strategies used for each candidate",  
  "candidates": {
    "candidate_1": "shortened version1",
    "candidate_2": "shortened version2",  
    "candidate_3": "shortened version3",
    "candidate_4": "shortened version4",
    "candidate_5": "shortened version5"
  }
  }

In [3]:
type(x["candidates"])

dict